In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import plotly.graph_objects as go

from datetime import timedelta
from utils.DataProcess import Start
from plotly.graph_objs.layout import YAxis,XAxis,Margin

In [2]:
df = Start('HL_14')

In [3]:
layout = go.Layout(
    # title="{}".format(self.df_last.Datetime),
    yaxis=YAxis(
        title="<b> system KPI</b>"
    ),
    yaxis2 = YAxis(
        title="<b> Wet bulb temp </b>",
        overlaying= 'y', 
        side= 'right',
    ),
    xaxis=dict(
        title="<b> Cost (Kw)</b>"
    ),
)
fig =  go.Figure(layout=layout)
fig.add_trace(
    go.Scatter(
        x = df.Datetime,
        y = df.system_KPI,
        name = 'system KPI',
        mode='lines',
        yaxis='y2'
    ))
fig.add_trace(
    go.Scatter(
        x = df.Datetime,
        y = df.Wet_bulb_temp,
        mode='lines',
        name = 'Wet bulb temp',
        yaxis='y1'
        
    ))

WB = df.Wet_bulb_temp.iloc[-100]

xs = df[df.Wet_bulb_temp==WB].Datetime
for x in xs:
    fig.add_vrect(
        x0=x - timedelta(minutes=15),
        x1=x + timedelta(minutes=15),
        fillcolor="Green",
        opacity=0.5,
        line_width=0,
    )

fig.add_hline(y=df.Wet_bulb_temp.iloc[-1])


In [4]:
Ms,Ai,Max,Min,Mds = [],[],[],[],[]


for WB in np.linspace(40,80,41):

    before = df[df.Datetime < '2022/10/28']
    before.Enthalpy = round(before.Enthalpy,1)
    Ms.append(np.mean(before[before.Enthalpy==round(WB,0)].system_KPI))
    Max.append(before[before.Enthalpy==round(WB,0)].system_KPI.max())
    Min.append(before[before.Enthalpy==round(WB,0)].system_KPI.min())
    Mds.append(np.median(before[before.Enthalpy==round(WB,0)].system_KPI))

    after = df[df.Datetime >= '2022/10/29']
    after.Enthalpy = round(after.Enthalpy,1)
    ahter_rh = after[after.Enthalpy==round(WB,1)]
    Ai.append(np.mean(ahter_rh.system_KPI))

layout = go.Layout(
    # title="{}".format(self.df_last.Datetime),
    yaxis=YAxis(
        title="<b> system KPI</b>"
    ),
    xaxis=dict(
        title="<b>Enthalpy</b>"
    ),
)
fig =  go.Figure(layout=layout)
fig.add_trace(
    go.Scatter(
        x = np.linspace(40,80,41),
        y = Ms,
        name = 'before Mean',
        # mode='markers',
        opacity=0.7,
        marker=dict(
            # color='LightSkyBlue',
            size=3),
        line = dict(
            # color='royalblue', 
            width=4, 
            # dash='dash'
            )
    ))

fig.add_trace(
    go.Scatter(
        x = np.linspace(40,80,41),
        y = Mds,
        name = 'before Medain',
        # mode='markers',
        opacity=0.7,
        marker=dict(
            # color='LightSkyBlue',
            size=3),
        line = dict(
            # color='royalblue', 
            width=4, 
            # dash='dash'
            )
    ))

fig.add_trace(
    go.Scatter(
        x = np.linspace(40,80,41),
        y = Max,
        name = 'before Max',
        # mode='markers',
        opacity=0.7,
        marker=dict(
            # color='LightSkyBlue',
            size=3),
        line = dict(
            # color='royalblue', 
            # width=4, 
            dash='dash')
    ))


fig.add_trace(
    go.Scatter(
        x = np.linspace(40,80,41),
        y = Min,
        name = 'before Min',
        # mode='markers',
        opacity=0.7,
        marker=dict(
            # color='LightSkyBlue',
            size=3),
        line = dict(
            # color='royalblue', 
            # width=4, 
            dash='dash')
    ))

fig.add_trace(go.Scatter(
     x = np.linspace(40,80,41),
     y = Max,
    fill='tonexty',
    fillcolor='rgba(184, 247, 212,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Ideal',
))

fig.add_trace(
    go.Scatter(
        x = after.Enthalpy,
        y = after.system_KPI,
        mode='markers',
        opacity=1,
        marker=dict(
            color='red',
            size=3),
        # line = dict(
        #     # color='royalblue', 
        #     width=4, 
        #     # dash='dash'
        #     ),
            
        name = 'AI control',
        
    ))

In [5]:
Ms,Ai,Max,Min,Mds = [],[],[],[],[]


for WB in np.linspace(40,80,41):

    before = df[df.Datetime < '2022/10/28']
    before.Enthalpy = round(before.Enthalpy,1)
    Ms.append(np.mean(before[before.Enthalpy==round(WB,0)].system_KPI))
    Mds.append(np.median(before[before.Enthalpy==round(WB,0)].system_KPI))

    after = df[df.Datetime >= '2022/10/29']
    after.Enthalpy = round(after.Enthalpy,0)
    ahter_rh = after[after.Enthalpy==round(WB,0)]
    Ai.append(np.mean(ahter_rh.system_KPI))

index = np.where(np.array(Ai)!=np.NaN)
baseline = np.array(Ms)[index]
print('Mean KPI {0:.2f}%'.format(((np.nanmean(baseline - np.array(Ai))) / np.nanmean(baseline))*100))

index = np.where(np.array(Ai)!=np.NaN)
baseline = np.array(Mds)[index]
print('Medain KPI {0:.2f}%'.format(((np.nanmean(baseline - np.array(Ai))) / np.nanmean(baseline))*100))

Mean KPI 0.43%
Medain KPI 0.49%
